<a href="https://colab.research.google.com/github/powerhanks/adams_crome_crowll/blob/master/kakaojuso_asynco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import asyncio
import nest_asyncio

In [24]:
import aiohttp

In [44]:
import pandas as pd
from tenacity import retry, stop_after_attempt,wait_fixed

In [49]:
#API 호출 함수
@retry(stop=stop_after_attempt(3),wait=wait_fixed(10))  # 3번 재시도
async def send_data_to_api(session, url, headers, data,max_retries=5):
  for attempt in range(1, max_retries + 1):
    #print('3')
    try:
      async with session.get(url, headers=headers, params=data) as response:
        response.raise_for_status()
        result = await response.json()
        #print(response.raise_for_status())
        try:
          match_first = result['documents'][0]['address']
          #print(match_first)
          return data['query'], match_first['address_name'],float(match_first['x']), float(match_first['y']),'ok'
        except:
          match_first=[data['query'],'','','','ok']
          return match_first

    except aiohttp.ClientResponseError as e:
      # Log the HTTP error (e.g., 4xx or 5xx)
      print(f"HTTP error {e.status} while sending request for data: {data}")

      if e.status == 429 and attempt < max_retries:
        # HTTP 429 (rate limit exceeded) - retry with exponential backoff
        backoff_seconds = 2 ** attempt
        print(f"Retrying in {backoff_seconds} seconds...")
        await asyncio.sleep(backoff_seconds)
      else:
        match_first=[data['query'],'','','',e.status]
        return match_first

    except Exception as e:
      # Log other types of errors
      print(f"Error sending request for data {data}: {e}")
      match_first=[data['query'],'','','',e.status]
      return match_first

In [50]:

async def process_csv_data(csv_file, api_url, headers):
  #print('2')
  tasks = []
  # aiohttp 클라이언트 세션 생성
  async with aiohttp.ClientSession() as session:
    # CSV 파일 읽기
    df = pd.read_csv(csv_file,dtype=str,encoding='cp949')

    for index, row in df.iterrows():
        # Assuming your CSV has an 'address' column
        address = row.get('상위국주소')
        if address:
          data = {'query': address}
          tasks.append(send_data_to_api(session, api_url, headers, data))

    results = await asyncio.gather(*tasks)

  # Create a DataFrame from the results
  results_df = pd.DataFrame(results)
  #results_df = pd.DataFrame(results,columns=["result"])
  #results_df["original_data"] = df.to_dict(orient="records")
  results_df.to_csv('results.csv', sep=',', header=True, index=True, mode='w', encoding='UTF-8-sig')

  #return results

In [51]:
# 비동기 함수
async def main():
  print('1')
  csv_file = './juso.csv'
  api_url = 'https://dapi.kakao.com/v2/local/search/address.json'
  api_key = "4471f0e1677512343d70b7852d768e3e"
  headers = {"Authorization": "KakaoAK " + api_key}

  #results = await process_csv_data(csv_file, api_url, headers)
  await process_csv_data(csv_file, api_url, headers)
  # Process results as needed
  #print(results)

In [52]:
# asyncio를 실행시킴
if __name__ == "__main__":
  print('0')
  nest_asyncio.apply()
  asyncio.run(main())
  """
  # Run the main coroutine using create_task
  asyncio.create_task(main())

  # Keep the event loop running
  asyncio.get_event_loop().run_forever()
  """

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
HTTP error 429 while sending request for data: {'query': '광주 서구 시청로 30(치평동)'}
Retrying in 4 seconds...
HTTP error 429 while sending request for data: {'query': '제주 서귀포시 성산읍 신양로 103(고성리)'}
Retrying in 4 seconds...
HTTP error 429 while sending request for data: {'query': '전남 영암군 삼호읍 대불산단8로1길 35(난전리)'}
Retrying in 4 seconds...
HTTP error 429 while sending request for data: {'query': '전북 부안군 부안읍 봉덕리222-1번지'}
Retrying in 4 seconds...
HTTP error 429 while sending request for data: {'query': '전북 남원시 금동로 23(금동)'}
Retrying in 4 seconds...
HTTP error 429 while sending request for data: {'query': '전남 목포시 통일대로 66(상동)'}
Retrying in 4 seconds...
HTTP error 429 while sending request for data: {'query': '전남 강진군 신전면 용월리678-17번지'}
Retrying in 4 seconds...
HTTP error 429 while sending request for data: {'query': '전남 목포시 청호로 109(산정동)'}
Retrying in 4 seconds...
HTTP error 429 while sending request for data: {'query': '광주 서구 무진대로 937(광천동)'}
Retrying in 4 seconds...
HTTP e